## imports

In [1]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import stanza
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [48]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.model_selection import train_test_split

## Selection Data par langues

In [13]:
data = pd.read_csv('../data/train_data_prep_v1.csv')

In [14]:
data

,pair_id,pair_lang,source_url_1,publish_date_1,source_url_2,publish_date_2,title_1,text_1,meta_description_1,meta_keywords_1,...,text_2,meta_description_2,meta_keywords_2,Geography,Entities,Time,Narrative,Overall,Style,Tone
0,1484084337_1484110209,en_en,https://www.washingtonpost.com,Wed Jan 1 00:00:00 2020,https://www.washingtonpost.com,Wed Jan 1 00:00:00 2020,Virginia man arrested in fatal DUI crash in We...,"MARTINSBURG, W.Va. — A suspected drunken drive...",Police in West Virginia say a suspected drunke...,"['Highway Fatal-DUI-West Virginia', 'Martinsbu...",...,"PORT-AU-PRINCE, Haiti — Haitian President Jove...",Haitian President Jovenel Moïse has broken wit...,"['CB-Haiti-Political Turmoil', 'Jean', 'Haiti'...",4.0,4.000000,1.000000,4.000000,4.000000,1.666667,2.000000
1,1484396422_1483924666,en_en,https://www.stlucianewsonline.com,Wed Jan 1 21:17:15 2020,https://www.thestar.com,Wed Jan 1 00:00:00 2020,Guyana: Three injured after car crashes into u...,Share This On:\n\nPin 11 Shares\n\n(NEWS ROOM ...,-,[''],...,BERLIN - A fire at a zoo in western Germany in...,BERLIN - A fire at a zoo in western Germany in...,"['smg2_world', 'smg_europe', 'smg2_news']",4.0,4.000000,1.000000,4.000000,3.666667,1.666667,1.333333
2,1484698254_1483758694,en_en,https://www.teaparty.org,NaN,https://www.timesofisrael.com,NaN,Trump Brings In 2020 At Mar-a-Lago: ‘We’re Goi...,(Breitbart) – President Donald Trump welcomed ...,NaN,[''],...,"PALM BEACH, United States — US President Donal...",US president says response to rioting by pro-I...,[''],1.0,2.000000,1.000000,2.333333,2.333333,1.000000,1.333333
3,1576314516_1576455088,en_en,https://gadgets.ndtv.com,NaN,https://gadgets.ndtv.com,NaN,Zomato Buys Uber's Food Delivery Business in I...,Uber has sold its online food-ordering busines...,Zomato on Tuesday announced it has acquired Ub...,['zomato uber eats business acquisition india ...,...,Rapid digitisation and growth in both online b...,The report by Google and Boston Consulting Gro...,['indian online food delivery market to hit us...,1.0,2.333333,2.666667,1.666667,2.000000,1.666667,1.666667
4,1484036253_1483894099,en_en,https://news.yahoo.com,Wed Jan 1 08:57:59 2020,NaN,NaN,"India approves third moon mission, months afte...",BENGALURU (Reuters) - India has approved its t...,India has approved its third lunar mission mon...,[''],...,NaN,NaN,NaN,1.0,1.250000,1.000000,1.250000,1.250000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4959,1586195445_1598778991,tr_tr,http://www.haberler.com,NaN,https://www.haberler.com,NaN,"BM, Aden'de 2 bini aşkın iç göçmenin selden za...","BM, Aden'de 2 bini aşkın iç göçmenin selden za...","Birleşmiş Milletler (BM), 2 bini aşkın iç göçm...","['Birleşmiş Milletler', 'Twitter', 'Yemen', 'G...",...,BM'den Yemen'de kadınların doğumda ölüm riski ...,BİRLEŞMİŞ Birleşmiş Milletler (BM) dünyanın en...,"['Birleşmiş Milletler', 'Yemen', 'Güncel', 'Ha...",1.0,2.000000,2.000000,4.000000,3.000000,1.000000,1.000000
4960,1590915424_1590940388,tr_tr,https://www.haberler.com,NaN,https://www.aksam.com.tr,NaN,Kovid-19'dan dolayı La Liga kulüplerinde hayat...,Kovid-19'dan dolayı La Liga kulüplerinde hayat...,Yeni tip koronavirüs (Kovid-19) salgınının eko...,"['Real Madrid', 'İspanya', 'La Liga', 'Futbol'...",...,Yeni tip koronavirüs (Kovid-19) salgınının eko...,Yeni tip koronavirüs (Kovid-19) salgınının eko...,"['İspanya 1. Futbol Ligi', 'la liga', 'koronav...",1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
4961,1526157103_1492737005,tr_tr,http://www.samanyoluhaber.com,Thu Feb 20 10:47:10 2020,https://www.fotomac.com.tr,Sun Jan 12 00:00:00 2020,Saray da çare olmadı: 'Borca boğulan dev kulüp...,\n\n\n\n\n\n\n\nİflas noktasındaki kulüplerin ...,SARAY DA ÇARE OLMADI: 'BORCA BOĞULAN DEV KULÜP...,"['Saray', 'da', 'çare', 'olmadı:', ""'Borca"", '...",...,"TFF, resmi internet sitesinden Beşiktaş'ın fai...","Federasyon, Başkan Çebi’nin yaptığı açıklamala...",[''],1.0,2.000000,3.0

In [5]:
# remémorer numéro de ligne - compléter les Nan
#data['ligne'] = data.index
#data = data.fillna('')

In [20]:
# séparation des datasets
espagnol = data.loc[data.pair_lang == 'es_es',['title_1','title_2','text_1','text_2','Geography', 'Entities',
       'Time', 'Narrative', 'Overall', 'Style', 'Tone']].reset_index(drop=True)

In [21]:
espagnol.head(10)

,title_1,title_2,text_1,text_2,Geography,Entities,Time,Narrative,Overall,Style,Tone
0,Se realizan compras de última hora para fin de...,Heridos de pirotecnia,"PROGRESO, Yucatán, 31 de diciembre.- Cuando al...","Este miércoles a la madrugada, en el marco de ...",4.0,4.0,1.0,4.0,4.0,1.0,2.0
1,Realizarán obras de mejoramiento en el Bosque ...,Arica: Con ambiente festivo se inauguró la últ...,Las obras iniciarán durante el primer trimestr...,"En la Población Tarapacá Oriente, los vecinos ...",3.0,3.5,1.0,2.5,3.0,1.0,1.0
2,Estos son los países más peligrosos para viaja...,Condena FIP 49 asesinatos de periodistas en 2019,Es probable que uno de los propósitos más repe...,Los 49 asesinatos de este año ocurrieron en un...,2.0,4.0,3.0,4.0,3.0,2.0,1.0
3,Un niño sufrió quemaduras luego de poner una c...,Australia recibe el año 2020 con devastadores ...,01/01/2020 - 12:32 Policiales\n\nEn las última...,"Según los medios, más de 200 viviendas fueron ...",4.0,4.0,1.0,4.0,4.0,1.0,1.0
4,Francisco pidió «disculpas» por el incidente c...,El Papa denuncia la violencia contra las mujer...,El papa Francisco pidió hoy «disculpas» por el...,"En su primera misa del año 2020, el Papa Franc...",1.0,1.0,1.0,3.0,2.0,2.0,2.0
5,"Multan a hijo de payaso ""Chimbombín"" en Comala...",Policía multa al hijo del payaso Chimbombín po...,"El hijo de Douglas Hernández Fajardo, conocido...",El padre intercedió para que su hijo no fuera ...,1.0,1.0,1.0,1.0,1.0,1.0,1.0
6,NaN,NaN,NaN,NaN,2.0,4.0,2.0,4.0,4.0,1.0,3.0
7,Los primeros bebés de 2020 y la madre que dio ...,En Rosario las primeras bebas del 2020 fueron ...,"Martina, la hija de una mujer platense que est...",Ambas nacieron durante las primeras horas del ...,2.0,3.0,1.0,3.0,2.0,2.0,1.0
8,Para comenzar bien el año: el golazo de chilen...,"Una chilena en la Premier League, el primer go...",¿encontraste un error? avísanos\n\nEnvíanos tu...,Este año nuevo 2020 empezó con mucha actividad...,1.0,3.0,1.0,1.0,1.0,3.0,2.0
9,Respirando mina inspirado por León,Las nocheviejas infantiles triunfan,"Samuel Teijido Fraga, un gallego de 26 años af...",Las nocheviejas infantiles triunfan y se conso...,3.0,4.0,1.0,4.0,4.0,1.0,1.0


In [22]:
espagnol.shape

(570, 11)

In [23]:
espagnol = espagnol.dropna()

In [24]:
display(espagnol.shape)
espagnol.head(10)

(490, 11)

,title_1,title_2,text_1,text_2,Geography,Entities,Time,Narrative,Overall,Style,Tone
0,Se realizan compras de última hora para fin de...,Heridos de pirotecnia,"PROGRESO, Yucatán, 31 de diciembre.- Cuando al...","Este miércoles a la madrugada, en el marco de ...",4.0,4.0,1.0,4.0,4.0,1.0,2.0
1,Realizarán obras de mejoramiento en el Bosque ...,Arica: Con ambiente festivo se inauguró la últ...,Las obras iniciarán durante el primer trimestr...,"En la Población Tarapacá Oriente, los vecinos ...",3.0,3.5,1.0,2.5,3.0,1.0,1.0
2,Estos son los países más peligrosos para viaja...,Condena FIP 49 asesinatos de periodistas en 2019,Es probable que uno de los propósitos más repe...,Los 49 asesinatos de este año ocurrieron en un...,2.0,4.0,3.0,4.0,3.0,2.0,1.0
3,Un niño sufrió quemaduras luego de poner una c...,Australia recibe el año 2020 con devastadores ...,01/01/2020 - 12:32 Policiales\n\nEn las última...,"Según los medios, más de 200 viviendas fueron ...",4.0,4.0,1.0,4.0,4.0,1.0,1.0
4,Francisco pidió «disculpas» por el incidente c...,El Papa denuncia la violencia contra las mujer...,El papa Francisco pidió hoy «disculpas» por el...,"En su primera misa del año 2020, el Papa Franc...",1.0,1.0,1.0,3.0,2.0,2.0,2.0
5,"Multan a hijo de payaso ""Chimbombín"" en Comala...",Policía multa al hijo del payaso Chimbombín po...,"El hijo de Douglas Hernández Fajardo, conocido...",El padre intercedió para que su hijo no fuera ...,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7,Los primeros bebés de 2020 y la madre que dio ...,En Rosario las primeras bebas del 2020 fueron ...,"Martina, la hija de una mujer platense que est...",Ambas nacieron durante las primeras horas del ...,2.0,3.0,1.0,3.0,2.0,2.0,1.0
8,Para comenzar bien el año: el golazo de chilen...,"Una chilena en la Premier League, el primer go...",¿encontraste un error? avísanos\n\nEnvíanos tu...,Este año nuevo 2020 empezó con mucha actividad...,1.0,3.0,1.0,1.0,1.0,3.0,2.0
9,Respirando mina inspirado por León,Las nocheviejas infantiles triunfan,"Samuel Teijido Fraga, un gallego de 26 años af...",Las nocheviejas infantiles triunfan y se conso...,3.0,4.0,1.0,4.0,4.0,1.0,1.0
10,Guadalajara: cuatro hermanos heridos tras riña...,Fiesta de Año Nuevo termina en riña familiar,Juan Carlos Munguía\n\nCuatro hermanos resulta...,Una fiesta familiar familiar de Año Nuevo term...,1.0,1.0,1.0,1.0,1.0,1.0,1.0


## Modèles

In [25]:
#model = SentenceTransformer("sentence-transformers/distiluse-base-multilingual-cased-v2")
#model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased')
#model = SentenceTransformer("https://public.ukp.informatik.tu-darmstadt.de/reimers/sentence-transformers/v0.2/distiluse-base-multilingual-cased.zip")
model = SentenceTransformer("sentence-transformers/distiluse-base-multilingual-cased-v2")

In [26]:
nlp = stanza.Pipeline(lang='es', processors='tokenize')

2022-01-03 10:01:11 WARNING: Language es package default expects mwt, which has been added
2022-01-03 10:01:11 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |

2022-01-03 10:01:11 INFO: Use device: cpu
2022-01-03 10:01:11 INFO: Loading: tokenize
2022-01-03 10:01:11 INFO: Loading: mwt
2022-01-03 10:01:11 INFO: Done loading processors!


## Sentence transformers doc vector

In [27]:
def centroidEmbeddings(embeddings):
    #print("embed=", len(embeddings))
    SommeEmb= [0]*512
    nbreEmb= len(embeddings)
    if nbreEmb > 0:
        for phrase in embeddings:
            #print("phrase=",len(phrase))
            SommeEmb+= phrase
        #print(SommeEmb / nbreEmb)
        return SommeEmb / nbreEmb
    else:
        return []

In [28]:
def comparaisonCentroidEmbeddings(embeddings1, embeddings2):
    centroidEmb1 = centroidEmbeddings(embeddings1)
    centroidEmb2 = centroidEmbeddings(embeddings2)
    if not(len(centroidEmb1) == 0) and not(len(centroidEmb2) == 0):
        diff= (centroidEmb2 - centroidEmb1)
    else: 
        diff = [-1]
    return diff

In [29]:
def sentencesTransformersSimDiffRow(row, modelSentenceTrans, modelStanza):
    #print("ligne", row['pair_id'].values)
    return sentencesTransformersSimDiff(row['title_1'] + row['text_1'], row['title_2'] + row['text_2'], modelSentenceTrans, modelStanza)
    
def sentencesTransformersSimDiff(texte1,texte2, modelSentenceTrans, modelStanza):
    #print(texte1, texte2)
    doc1 = modelStanza(texte1)
    sentences1 = [sentence.text for sentence in doc1.sentences]
    taille= len(sentences1)
    #print('taille liste=', taille)
    embeddings1 = modelSentenceTrans.encode(sentences1)
    #print(embeddings1)
    #centroidEmb1 = centroidEmbeddings(embeddings1)
    
    
    doc2 = modelStanza(texte2)
    sentences2 = [sentence.text for sentence in doc2.sentences]#doc2.sentences
    embeddings2 = modelSentenceTrans.encode(sentences2)
    #print(embeddings2)
    #return cosine_similarity(embeddings1, embeddings2)
    #centroidEmb2 = centroidEmbeddings(embeddings2)
    return comparaisonCentroidEmbeddings(embeddings1, embeddings2)

In [30]:
df_cos = pd.DataFrame()
for index, ligne in espagnol.iterrows():
    print("index= ",index)
    diff = sentencesTransformersSimDiffRow(ligne, model, nlp)
    #print(diff)
    ligne['vectSim'] = diff
    df_cos =df_cos.append(ligne, ignore_index= True)

index=  0


C:\Users\camille.gosset\Anaconda3\lib\site-packages\stanza\models\common\beam.py:86: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  prevK = bestScoresId // numWords


index=  1
index=  2
index=  3
index=  4
index=  5
index=  7
index=  8
index=  9
index=  10
index=  11
index=  12
index=  13
index=  14
index=  16
index=  18
index=  19
index=  20
index=  21
index=  22
index=  23
index=  24
index=  25
index=  26
index=  27
index=  29
index=  30
index=  31
index=  32
index=  33
index=  34
index=  35
index=  36
index=  37
index=  38
index=  40
index=  41
index=  42
index=  43
index=  44
index=  46
index=  48
index=  50
index=  52
index=  53
index=  54
index=  55
index=  56
index=  57
index=  58
index=  59
index=  62
index=  63
index=  64
index=  65
index=  66
index=  67
index=  69
index=  70
index=  71
index=  72
index=  73
index=  75
index=  78
index=  79
index=  80
index=  82
index=  83
index=  85
index=  86
index=  87
index=  88
index=  89
index=  90
index=  91
index=  93
index=  94
index=  95
index=  97
index=  98
index=  99
index=  100
index=  101
index=  102
index=  103
index=  104
index=  105
index=  106
index=  107
index=  108
index=  109
index=  

In [31]:
df_cos.head(7)

,Entities,Geography,Narrative,Overall,Style,Time,Tone,text_1,text_2,title_1,title_2,vectSim
0,4.0,4.0,4.0,4.0,1.0,1.0,2.0,"PROGRESO, Yucatán, 31 de diciembre.- Cuando al...","Este miércoles a la madrugada, en el marco de ...",Se realizan compras de última hora para fin de...,Heridos de pirotecnia,"[-0.021382593986345455, -0.015521107509266585,..."
1,3.5,3.0,2.5,3.0,1.0,1.0,1.0,Las obras iniciarán durante el primer trimestr...,"En la Población Tarapacá Oriente, los vecinos ...",Realizarán obras de mejoramiento en el Bosque ...,Arica: Con ambiente festivo se inauguró la últ...,"[0.0064300460265561315, -0.007713928256028643,..."
2,4.0,2.0,4.0,3.0,2.0,3.0,1.0,Es probable que uno de los propósitos más repe...,Los 49 asesinatos de este año ocurrieron en un...,Estos son los países más peligrosos para viaja...,Condena FIP 49 asesinatos de periodistas en 2019,"[-0.025938336395055932, -0.018853332136880934,..."
3,4.0,4.0,4.0,4.0,1.0,1.0,1.0,01/01/2020 - 12:32 Policiales\n\nEn las última...,"Según los medios, más de 200 viviendas fueron ...",Un niño sufrió quemaduras luego de poner una c...,Australia recibe el año 2020 con devastadores ...,"[0.03831898104787494, 0.035793492337688804, 0...."
4,1.0,1.0,3.0,2.0,2.0,1.0,2.0,El papa Francisco pidió hoy «disculpas» por el...,"En su primera misa del año 2020, el Papa Franc...",Francisco pidió «disculpas» por el incidente c...,El Papa denuncia la violencia contra las mujer...,"[0.05061872240641755, -0.016140417521668282, 0..."
5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,"El hijo de Douglas Hernández Fajardo, conocido...",El padre intercedió para que su hijo no fuera ...,"Multan a hijo de payaso ""Chimbombín"" en Comala...",Policía multa al hijo del payaso Chimbombín po...,"[-0.004512161936487674, 0.0038663446964004406,..."
6,3.0,2.0,3.0,2.0,2.0,1.0,1.0,"Martina, la hija de una mujer platense que est...",Ambas nacieron durante las primeras horas del ...,Los primeros bebés de 2020 y la madre que dio ...,En Rosario las primeras bebas del 2020 fueron ...,"[-0.012621174682863057, -0.005358497637974902,..."


In [32]:
def comparaison1phraseA1(embeddings1, embeddings2):
    #print("embed=", len(embeddings))
    SommeEmb= [0]*512
    nbreEmb = 0
    for phraseEmb1 in embeddings1:
        for phraseEmb2 in embeddings2:
            differenceEmbds= phraseEmb2 - phraseEmb1
            SommeEmb+= differenceEmbds
            nbreEmb+= 1
    if nbreEmb > 0:
        return SommeEmb / nbreEmb
    else: 
        return [-1]

In [33]:
def comparaisonAjoutZero(embeddings1, embeddings2):
    tailleEmb1 = len(embeddings1)
    tailleEmb2 = len(embeddings2)
    if (tailleEmb1 ==0) or (tailleEmb2 == 0):
        return [-1]
    if (tailleEmb1 == tailleEmb2):
        return embeddings2 - embeddings1
    else:
        bonSens= False
        if (tailleEmb1 > tailleEmb2):
            embeddingsPlusGrand= embeddings1
            embeddingsPlusPetit = embeddings2
            bonSens = True
        else:
            embeddingsPlusGrand= embeddings2
            embeddingsPlusPetit = embeddings1
        #embeddingsPlusPetit = np.concatenate(embeddingsPlusPetit, np.array([0]*512))
        #embeddingsPlusPetit = np.concatenate(embeddingsPlusPetit, np.array([0]*512))
        #print("taille= ", len(embeddingsPlusPetit))
        embeddingsPlusPetit.resize((len(embeddingsPlusGrand), 512), refcheck=False)
        #print(embeddingsPlusPetit)
        if bonSens:
            return embeddingsPlusPetit - embeddingsPlusGrand
        else:
            return embeddingsPlusGrand - embeddingsPlusPetit

In [34]:
def comparaisonFlux(embeddings1, embeddings2):
    if (len(embeddings1) ==0) or (len(embeddings2) == 0):
        return [-1]
    if (len(embeddings1) == len(embeddings2)):
        return embeddings2 - embeddings1
    else:
        if (len(embeddings1) > len(embeddings2)):
            partage = (embeddings2 - embeddings1[:len(embeddings2)]) + (embeddings2 - embeddings1[len(embeddings1) - len(embeddings2):])
            return partage / 2
        else:
            partage = (embeddings2[:len(embeddings2)] - embeddings1) + (embeddings2[len(embeddings2) - len(embeddings1):] - embeddings1)
            return partage / 2

In [35]:
def sentencesTransformersFewMethodsRow(row, modelSentenceTrans, modelStanza):
    comparaison1a1, compAjoutZero, CompFlux = sentencesTransformersFewMethods(row['title_1'] + row['text_1'], row['title_2'] + row['text_2'], modelSentenceTrans, modelStanza)
    return comparaison1a1, compAjoutZero, CompFlux
    
def sentencesTransformersFewMethods(texte1,texte2, modelSentenceTrans, modelStanza):
    #print(texte1, texte2)
    doc1 = modelStanza(texte1)
    sentences1 = [sentence.text for sentence in doc1.sentences]
    embeddings1 = modelSentenceTrans.encode(sentences1)
    centroidEmb1 = centroidEmbeddings(embeddings1)
    
    
    doc2 = modelStanza(texte2)
    sentences2 = [sentence.text for sentence in doc2.sentences]#doc2.sentences
    embeddings2 = modelSentenceTrans.encode(sentences2)
    centroidEmb2 = centroidEmbeddings(embeddings2)
    
    compa1a1 = comparaison1phraseA1(embeddings1, embeddings2)
    compaAjoutZero = comparaisonAjoutZero(embeddings1, embeddings2)
    compaFlux = comparaisonFlux(embeddings1, embeddings2)
    return compa1a1, compaAjoutZero, compaFlux

In [36]:
def applicationFewMethods(df_cos):
    df_cosFinal = pd.DataFrame()
    for index, ligne in df_cos.iterrows():
        print("index= ",index)
        compaEachPhrase, comparaisonAjoutZero ,comparaisonFlux = sentencesTransformersFewMethodsRow(ligne, model, nlp)
        #print(diff)
        ligne['compaEachPhrase'] = compaEachPhrase
        ligne['comparaisonAjoutZero'] = comparaisonAjoutZero
        ligne['comparaisonFlux'] = comparaisonFlux
        df_cosFinal =df_cosFinal.append(ligne, ignore_index= True)
    return df_cosFinal

In [37]:
df_cosFinal = applicationFewMethods(df_cos)

index=  0


C:\Users\camille.gosset\Anaconda3\lib\site-packages\stanza\models\common\beam.py:86: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  prevK = bestScoresId // numWords


index=  1
index=  2
index=  3
index=  4
index=  5
index=  6
index=  7
index=  8
index=  9
index=  10
index=  11
index=  12
index=  13
index=  14
index=  15
index=  16
index=  17
index=  18
index=  19
index=  20
index=  21
index=  22
index=  23
index=  24
index=  25
index=  26
index=  27
index=  28
index=  29
index=  30
index=  31
index=  32
index=  33
index=  34
index=  35
index=  36
index=  37
index=  38
index=  39
index=  40
index=  41
index=  42
index=  43
index=  44
index=  45
index=  46
index=  47
index=  48
index=  49
index=  50
index=  51
index=  52
index=  53
index=  54
index=  55
index=  56
index=  57
index=  58
index=  59
index=  60
index=  61
index=  62
index=  63
index=  64
index=  65
index=  66
index=  67
index=  68
index=  69
index=  70
index=  71
index=  72
index=  73
index=  74
index=  75
index=  76
index=  77
index=  78
index=  79
index=  80
index=  81
index=  82
index=  83
index=  84
index=  85
index=  86
index=  87
index=  88
index=  89
index=  90
index=  91
index=  

In [38]:
df_cosFinal.head(10)

,Entities,Geography,Narrative,Overall,Style,Time,Tone,compaEachPhrase,comparaisonAjoutZero,comparaisonFlux,text_1,text_2,title_1,title_2,vectSim
0,4.0,4.0,4.0,4.0,1.0,1.0,2.0,"[-0.021382593891466968, -0.015521107487147674,...","[[-0.03819096, -0.0065030716, -0.07167914, -0....","[[-0.03819096, -0.0065030716, -0.07167914, -0....","PROGRESO, Yucatán, 31 de diciembre.- Cuando al...","Este miércoles a la madrugada, en el marco de ...",Se realizan compras de última hora para fin de...,Heridos de pirotecnia,"[-0.021382593986345455, -0.015521107509266585,..."
1,3.5,3.0,2.5,3.0,1.0,1.0,1.0,"[0.006430046438860397, -0.0077139282269248115,...","[[-0.060858622, 0.030139182, -0.005321129, -0....","[[-0.060858622, 0.030139182, -0.005321129, -0....",Las obras iniciarán durante el primer trimestr...,"En la Población Tarapacá Oriente, los vecinos ...",Realizarán obras de mejoramiento en el Bosque ...,Arica: Con ambiente festivo se inauguró la últ...,"[0.0064300460265561315, -0.007713928256028643,..."
2,4.0,2.0,4.0,3.0,2.0,3.0,1.0,"[-0.025938336573102894, -0.01885333186125054, ...","[[-0.012110677, -0.0058718347, -0.105579674, -...","[[-0.012110677, -0.0058718347, -0.105579674, -...",Es probable que uno de los propósitos más repe...,Los 49 asesinatos de este año ocurrieron en un...,Estos son los países más peligrosos para viaja...,Condena FIP 49 asesinatos de periodistas en 2019,"[-0.025938336395055932, -0.018853332136880934,..."
3,4.0,4.0,4.0,4.0,1.0,1.0,1.0,"[0.03831898128070558, 0.035793492337688804, 0....","[[-0.038013734, -0.006293688, 0.010512967, 0.0...","[[-0.038013734, -0.006293688, 0.010512967, 0.0...",01/01/2020 - 12:32 Policiales\n\nEn las última...,"Según los medios, más de 200 viviendas fueron ...",Un niño sufrió quemaduras luego de poner una c...,Australia recibe el año 2020 con devastadores ...,"[0.03831898104787494, 0.035793492337688804, 0...."
4,1.0,1.0,3.0,2.0,2.0,1.0,2.0,"[0.05061872253694382, -0.016140417441790237, 0...","[[0.056674074, 0.0043663415, 0.030819347, 0.01...","[[0.056674074, 0.0043663415, 0.030819347, 0.01...",El papa Francisco pidió hoy «disculpas» por el...,"En su primera misa del año 2020, el Papa Franc...",Francisco pidió «disculpas» por el incidente c...,El Papa denuncia la violencia contra las mujer...,"[0.05061872240641755, -0.016140417521668282, 0..."
5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,"[-0.004512161937514868, 0.0038663446928052614,...","[[0.0048376694, 0.013151023, 0.029229729, 0.00...","[[0.0048376694, 0.013151023, 0.029229729, 0.00...","El hijo de Douglas Hernández Fajardo, conocido...",El padre intercedió para que su hijo no fuera ...,"Multan a hijo de payaso ""Chimbombín"" en Comala...",Policía multa al hijo del payaso Chimbombín po...,"[-0.004512161936487674, 0.0038663446964004406,..."
6,3.0,2.0,3.0,2.0,2.0,1.0,1.0,"[-0.012621174585795168, -0.0053584977636505346...","[[-0.045812473, 0.028829396, 0.017955702, -0.0...","[[-0.045812473, 0.028829396, 0.017955702, -0.0...","Martina, la hija de una mujer platense que est...",Ambas nacieron durante las primeras horas del ...,Los primeros bebés de 2020 y la madre que dio ...,En Rosario las primeras bebas del 2020 fueron ...,"[-0.012621174682863057, -0.005358497637974902,..."
7,3.0,1.0,1.0,1.0,3.0,1.0,2.0,"[0.011685577113114829, 0.024564656417905575, -...","[[0.051749174, 0.035588652, -0.022815108, -0.0...","[[0.051749174, 0.035588652, -0.022815108, -0.0...",¿encontraste un error? avísanos\n\nEnvíanos tu...,Este año nuevo 2020 empezó con mucha actividad...,Para comenzar bien el año: el golazo de chilen...,"Una chilena en la Premier League, el primer go...","[0.011685577150880514, 0.024564656445623508, -..."
8,4.0,3.0,4.0,4.0,1.0,1.0,1.0,"[-0.0574516240764075, -0.004352958519937089, 0...","[[-0.12472447, 0.054585587, -0.04314705, 0.004...","[[-0.12472447, 0.054585587, -0.04314705, 0.004...","Samuel Teijido Fraga, un gallego de 26 años af...",Las nocheviejas infantiles triunfan y se conso...,Respirando mina inspirado por León,Las nocheviejas infantiles triunfan,"

In [56]:
df_cosFinal['Overall'] = df_cosFinal['Overall'].astype('int32')

In [57]:
df_cosFinal.dtypes

Entities                float64
Geography               float64
Narrative               float64
Overall                   int32
Style                   float64
Time                    float64
Tone                    float64
compaEachPhrase          object
comparaisonAjoutZero     object
comparaisonFlux          object
text_1                   object
text_2                   object
title_1                  object
title_2                  object
vectSim                  object
dtype: object

In [59]:
df_cosFinal = df_cosFinal.dropna()

In [60]:
df_cosFinal.head(10)

,Entities,Geography,Narrative,Overall,Style,Time,Tone,compaEachPhrase,comparaisonAjoutZero,comparaisonFlux,text_1,text_2,title_1,title_2,vectSim
0,4.0,4.0,4.0,4.0,1.0,1.0,2.0,"[-0.021382593891466968, -0.015521107487147674,...","[[-0.03819096, -0.0065030716, -0.07167914, -0....","[[-0.03819096, -0.0065030716, -0.07167914, -0....","PROGRESO, Yucatán, 31 de diciembre.- Cuando al...","Este miércoles a la madrugada, en el marco de ...",Se realizan compras de última hora para fin de...,Heridos de pirotecnia,"[-0.021382593986345455, -0.015521107509266585,..."
1,3.5,3.0,2.5,3.0,1.0,1.0,1.0,"[0.006430046438860397, -0.0077139282269248115,...","[[-0.060858622, 0.030139182, -0.005321129, -0....","[[-0.060858622, 0.030139182, -0.005321129, -0....",Las obras iniciarán durante el primer trimestr...,"En la Población Tarapacá Oriente, los vecinos ...",Realizarán obras de mejoramiento en el Bosque ...,Arica: Con ambiente festivo se inauguró la últ...,"[0.0064300460265561315, -0.007713928256028643,..."
2,4.0,2.0,4.0,3.0,2.0,3.0,1.0,"[-0.025938336573102894, -0.01885333186125054, ...","[[-0.012110677, -0.0058718347, -0.105579674, -...","[[-0.012110677, -0.0058718347, -0.105579674, -...",Es probable que uno de los propósitos más repe...,Los 49 asesinatos de este año ocurrieron en un...,Estos son los países más peligrosos para viaja...,Condena FIP 49 asesinatos de periodistas en 2019,"[-0.025938336395055932, -0.018853332136880934,..."
3,4.0,4.0,4.0,4.0,1.0,1.0,1.0,"[0.03831898128070558, 0.035793492337688804, 0....","[[-0.038013734, -0.006293688, 0.010512967, 0.0...","[[-0.038013734, -0.006293688, 0.010512967, 0.0...",01/01/2020 - 12:32 Policiales\n\nEn las última...,"Según los medios, más de 200 viviendas fueron ...",Un niño sufrió quemaduras luego de poner una c...,Australia recibe el año 2020 con devastadores ...,"[0.03831898104787494, 0.035793492337688804, 0...."
4,1.0,1.0,3.0,2.0,2.0,1.0,2.0,"[0.05061872253694382, -0.016140417441790237, 0...","[[0.056674074, 0.0043663415, 0.030819347, 0.01...","[[0.056674074, 0.0043663415, 0.030819347, 0.01...",El papa Francisco pidió hoy «disculpas» por el...,"En su primera misa del año 2020, el Papa Franc...",Francisco pidió «disculpas» por el incidente c...,El Papa denuncia la violencia contra las mujer...,"[0.05061872240641755, -0.016140417521668282, 0..."
5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,"[-0.004512161937514868, 0.0038663446928052614,...","[[0.0048376694, 0.013151023, 0.029229729, 0.00...","[[0.0048376694, 0.013151023, 0.029229729, 0.00...","El hijo de Douglas Hernández Fajardo, conocido...",El padre intercedió para que su hijo no fuera ...,"Multan a hijo de payaso ""Chimbombín"" en Comala...",Policía multa al hijo del payaso Chimbombín po...,"[-0.004512161936487674, 0.0038663446964004406,..."
6,4.0,2.0,4.0,4.0,1.0,2.0,3.0,[-1],[-1],[-1],,,,,[-1]
7,3.0,2.0,3.0,2.0,2.0,1.0,1.0,"[-0.012621174585795168, -0.0053584977636505346...","[[-0.045812473, 0.028829396, 0.017955702, -0.0...","[[-0.045812473, 0.028829396, 0.017955702, -0.0...","Martina, la hija de una mujer platense que est...",Ambas nacieron durante las primeras horas del ...,Los primeros bebés de 2020 y la madre que dio ...,En Rosario las primeras bebas del 2020 fueron ...,"[-0.012621174682863057, -0.005358497637974902,..."
8,3.0,1.0,1.0,1.0,3.0,1.0,2.0,"[0.011685577113114829, 0.024564656417905575, -...","[[0.051749174, 0.035588652, -0.022815108, -0.0...","[[0.051749174, 0.035588652, -0.022815108, -0.0...",¿encontraste un error? avísanos\n\nEnvíanos tu...,Este año nuevo 2020 empezó con mucha actividad...,Para comenzar bien el año: el golazo de chilen...,"Una chilena en la Premier League, el primer go...","[0.011685577150880514, 0.024564656445623508, -..."
9,4.0,3.0,4.0,4.0,1.0,1.0,1.0,"[-0.0574516240764075, -0.004352958519937089, 0...","[[-0.12472447, 0.054585587, -0.04314705, 0.004...","[[-0.12472447, 0.054585587, -0.04314705, 0.004...","Samuel Teijido Fraga, un gallego de 26 años af...",Las nocheviejas infantiles triunfan y se conso...,Respirando mina in

In [40]:
predicteurs = ['compaEachPhrase', 'comparaisonAjoutZero', 'comparaisonFlux', 'vectSim']

In [41]:
df_cosFinal['vectSim']

0      [-0.021382593986345455, -0.015521107509266585,...
1      [0.0064300460265561315, -0.007713928256028643,...
2      [-0.025938336395055932, -0.018853332136880934,...
3      [0.03831898104787494, 0.035793492337688804, 0....
4      [0.05061872240641755, -0.016140417521668282, 0...
                             ...                        
485    [0.027450400268814214, -0.005770127020595182, ...
486    [0.001726350643568568, -0.005866617326521214, ...
487    [-0.021869334498081698, -0.008492047041231259,...
488    [0.0170639633773438, 0.016973466869482458, 0.0...
489    [0.0321268206074213, 0.057883214511093684, 0.0...
Name: vectSim, Length: 490, dtype: object

In [75]:
# random Forest simple sur scikit learn
df_temp = df_cosFinal['vectSim'].reset_index(drop=True)
Ydataset = df_cosFinal['Overall'].reset_index(drop=True)

In [76]:
Xdataset = (df_temp.values).tolist()
#Xtrain = (df_cosFinal['vectSim'].values).tolist()
#Xtrain = np.array(list(map(np.float, Xtrain)))
#ytrain = ytrain.astype(int)
X_train, X_test, y_train, y_test = train_test_split(Xdataset, Ydataset, test_size=0.3, random_state=1)

In [80]:
y_test

353    3
107    4
165    4
197    4
324    3
      ..
351    1
95     4
122    3
23     4
13     4
Name: Overall, Length: 147, dtype: int32

In [77]:
clf= svm.SVC(probability=True)
clf.fit(X_train,y_train)

SVC(probability=True)

In [61]:
res_clf1 = clf.predict(X_test)

In [64]:
res_clf1

array([4, 4, 4, 4, 2, 1, 4, 4, 1, 1, 1, 4, 2, 2, 1, 1, 4, 1, 1, 4, 4, 1,
       1, 1, 2, 4, 4, 2, 4, 1, 4, 1, 4, 2, 1, 4, 2, 4, 1, 4, 1, 1, 4, 4,
       1, 1, 2, 1, 1, 2, 1, 1, 2, 4, 4, 1, 1, 1, 2, 2, 1, 4, 1, 1, 1, 2,
       2, 4, 1, 4, 1, 1, 1, 1, 4, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 4, 4, 1,
       4, 1, 2, 4, 4, 1, 4, 2, 1, 4, 4, 4, 1, 1, 1, 1, 2, 2, 1, 4, 4, 2,
       1, 1, 4, 4, 2, 4, 1, 4, 1, 4, 1, 4, 1, 1, 2, 1, 4, 4, 4, 4, 4, 4,
       1, 1, 1, 4, 2, 2, 1, 2, 4, 1, 1, 4, 4, 4, 4])

In [82]:
df_res_clf1 = pd.DataFrame()
df_res_clf1['X'] = X_test
df_res_clf1['Y'] = y_test.tolist()
df_res_clf1['SVM'] = res_clf1
df_res_clf1

,X,Y,SVM
0,"[0.00227321104870902, -0.019956916376637913, 0...",3,4
1,"[-0.04904913251927596, -0.0037335695302490868,...",4,4
2,"[-0.04739758467102157, -0.040090941715919017, ...",4,4
3,"[-0.03008002200107457, -0.0026295335688204927,...",4,4
4,"[0.030419631274223015, 0.008444546666093325, -...",3,2
...,...,...,...
142,"[-0.00439684325318555, -0.006152882021838224, ...",1,1
143,"[0.01947900192868529, -0.028656054330481726, 0...",4,4
144,"[0.03166113099238525, 0.013734298544780662, -0...",3,4
145,"[-0.026621177342142265, 0.017499411092460924, ...",4,4


In [83]:
dfequals = df_res_clf1[df_res_clf1.Y == df_res_clf1.SVM]
display(dfequals.count())

X      66
Y      66
SVM    66
dtype: int64

In [86]:
dfOneDiff = df_res_clf1[df_res_clf1.Y == (df_res_clf1.SVM - 1 )]
display(dfOneDiff.count())

X      21
Y      21
SVM    21
dtype: int64

In [88]:
dfOneDiff = df_res_clf1[df_res_clf1.Y == (df_res_clf1.SVM + 1 )]
display(dfOneDiff.count())

X      30
Y      30
SVM    30
dtype: int64

In [90]:
dfTwoDiff = df_res_clf1[df_res_clf1.Y == (df_res_clf1.SVM + 2 )]
display(dfTwoDiff.count())

X      19
Y      19
SVM    19
dtype: int64

In [91]:
dfTwoDiff = df_res_clf1[df_res_clf1.Y == (df_res_clf1.SVM - 2 )]
display(dfTwoDiff.count())

X      8
Y      8
SVM    8
dtype: int64

In [47]:
df_cosFinal.to_csv('../data/traitementsSoustractionVecteurDocv2.csv', sep = ';', index= False, encoding = 'utf-8')